### Importing packages

In [1]:
import numpy as np 
import pandas as pd

In [2]:
import pycountry_convert as pc

In [3]:
df = pd.read_csv("C02 Emissions Data.csv")

In [4]:
df.head()

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,1750,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63104 entries, 0 to 63103
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             63104 non-null  object 
 1   ISO 3166-1 alpha-3  61472 non-null  object 
 2   Year                63104 non-null  int64  
 3   Total               62904 non-null  float64
 4   Coal                21744 non-null  float64
 5   Oil                 21717 non-null  float64
 6   Gas                 21618 non-null  float64
 7   Cement              20814 non-null  float64
 8   Flaring             21550 non-null  float64
 9   Other               1620 non-null   float64
 10  Per Capita          18974 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 5.3+ MB


### Kontinent Variable erstellen

In [6]:
def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

In [7]:
#df["Country"].apply(country_to_continent)
# Manche Länder können von der Funktion nicht zugeordnet werden
# Jene kommen in eine Liste und der Kontinent wird "manuell" zugeordnet

In [8]:
countries_with_bad_label = ['Antarctica', 'Bonaire, Saint Eustatius and Saba', 'Faeroe Islands', 
                            'Micronesia (Federated States of)', 'Panama Canal Zone', 'French Equatorial Africa',
                            'French West Africa', "Kosovo", 'Kuwaiti Oil Fires', 'Leeward Islands',
                            'Occupied Palestinian Territory', 'Pacific Islands (Palau)','Ryukyu Islands' ,
                            'Saint Helena', 'Sint Maarten (Dutch part)', 'St. Kitts-Nevis-Anguilla', 'Timor-Leste',
                            'Wallis and Futuna Islands','International Transport', 'Global']

In [9]:
def country_to_continent2(country_name):
    if country_name in countries_with_bad_label:
        if country_name in ['Antarctica']:
            return 'Antarctica'
        elif country_name in ['Bonaire, Saint Eustatius and Saba', 'Panama Canal Zone', 'Leeward Islands']:
            return 'South America'
        elif country_name in ['Sint Maarten (Dutch part)', 'St. Kitts-Nevis-Anguilla']:
            return 'North America'
        elif country_name in ['Faeroe Islands',"Kosovo"]:
            return 'Europe'
        elif country_name in ['French Equatorial Africa', 'French West Africa', 'Saint Helena']:
            return 'Africa'
        elif country_name in ['Micronesia (Federated States of)', 'Pacific Islands (Palau)', 'Wallis and Futuna Islands']:
            return 'Oceania'
        elif country_name in ['Kuwaiti Oil Fires', 'Occupied Palestinian Territory', 'Ryukyu Islands', 'Timor-Leste']:
            return 'Asia'
        else:
            return None #International Transport and the Global measure are excluded
    else:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name

In [10]:
df["Continent"] = df["Country"].apply(country_to_continent2)

In [11]:
df["Continent"].unique()

array(['Asia', 'Europe', 'Africa', 'North America', 'Antarctica',
       'South America', 'Oceania', None], dtype=object)

### Country-Groups

#### OECD

In [12]:
from countrygroups import OECD

In [13]:
OECD[0]

'AUS'

In [14]:
def check_OECD(Country_name):
    #von den Ländern nicht erkannt werden ist keins in der OECD (selbes gilt für OPEC und G7)
    if Country_name in ['Antarctica', 'Bonaire, Saint Eustatius and Saba', 'Faeroe Islands', 
                   'Micronesia (Federated States of)', 'Panama Canal Zone', 'French Equatorial Africa',
                   'French West Africa', "Kosovo", 'Kuwaiti Oil Fires', 'Leeward Islands',
                   'Occupied Palestinian Territory', 'Pacific Islands (Palau)','Ryukyu Islands' ,
                   'Saint Helena', 'Sint Maarten (Dutch part)', 'St. Kitts-Nevis-Anguilla', 'Timor-Leste',
                   'Wallis and Futuna Islands']:
        return 0
    if Country_name in ['International Transport', 'Global']:
        return None
    elif pc.country_name_to_country_alpha3(Country_name) in OECD:
        return 1
    else:
        return 0

In [15]:
df["OECD"] = df["Country"].apply(check_OECD)

In [16]:
df[(df["OECD"] == 1) & (df["Year"] == 2006)]["Total"].sum()

13996.866354

In [17]:
df[(df["Country"] == "Global") & (df["Year"] == 2006)]["Total"]

63088    30593.116788
Name: Total, dtype: float64

#### OPEC

In [18]:
from countrygroups import OPEC

In [19]:
def check_OPEC(Country_name):
    if Country_name in ['Antarctica', 'Bonaire, Saint Eustatius and Saba', 'Faeroe Islands', 
                   'Micronesia (Federated States of)', 'Panama Canal Zone', 'French Equatorial Africa',
                   'French West Africa', "Kosovo", 'Kuwaiti Oil Fires', 'Leeward Islands',
                   'Occupied Palestinian Territory', 'Pacific Islands (Palau)','Ryukyu Islands' ,
                   'Saint Helena', 'Sint Maarten (Dutch part)', 'St. Kitts-Nevis-Anguilla', 'Timor-Leste',
                   'Wallis and Futuna Islands']:
        return 0
    if Country_name in ['International Transport', 'Global']:
        return None
    elif pc.country_name_to_country_alpha3(Country_name) in OPEC:
        return 1
    else:
        return 0

In [20]:
df["OPEC"] = df["Country"].apply(check_OPEC)

#### G7

In [21]:
from countrygroups import G7

In [22]:
def check_G7(Country_name):
    if Country_name in ['Antarctica', 'Bonaire, Saint Eustatius and Saba', 'Faeroe Islands', 
                   'Micronesia (Federated States of)', 'Panama Canal Zone', 'French Equatorial Africa',
                   'French West Africa', "Kosovo", 'Kuwaiti Oil Fires', 'Leeward Islands',
                   'Occupied Palestinian Territory', 'Pacific Islands (Palau)','Ryukyu Islands' ,
                   'Saint Helena', 'Sint Maarten (Dutch part)', 'St. Kitts-Nevis-Anguilla', 'Timor-Leste',
                   'Wallis and Futuna Islands']:
        return 0
    if Country_name in ['International Transport', 'Global']:
        return None
    elif pc.country_name_to_country_alpha3(Country_name) in G7:
        return 1
    else:
        return 0

In [23]:
df["G7"] = df["Country"].apply(check_G7)

### Decades and Centuries

#### Decades

In [24]:
from math import floor

In [25]:
def get_decade(Year):
    return (int(floor(Year / 10)) * 10) 

In [26]:
df["Decade"] = df["Year"].apply(get_decade)

In [27]:
df["Decade"].unique()

array([1750, 1760, 1770, 1780, 1790, 1800, 1810, 1820, 1830, 1840, 1850,
       1860, 1870, 1880, 1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960,
       1970, 1980, 1990, 2000, 2010, 2020], dtype=int64)

#### Centuries

In [28]:
def get_century(Year):
    if Year < 2000:
        return f"{int(floor(Year / 100) + 1)}th Century" 
    else:
        return "21st Century"

In [29]:
df["Century"] = df["Year"].apply(get_century)

In [30]:
df["Century"].unique()

array(['18th Century', '19th Century', '20th Century', '21st Century'],
      dtype=object)

## Exporting the data

In [31]:
# Float-format muss angepasst werden, weil SQL die Daten sonst falsch einliest
df.to_csv("C02EmissionData.csv", float_format = "%.6f") # Wide-format

## Changing to wide Format Aöpmg

In [32]:
df.columns

Index(['Country', 'ISO 3166-1 alpha-3', 'Year', 'Total', 'Coal', 'Oil', 'Gas',
       'Cement', 'Flaring', 'Other', 'Per Capita', 'Continent', 'OECD', 'OPEC',
       'G7', 'Decade', 'Century'],
      dtype='object')

In [33]:
df_long = pd.melt(df, id_vars = ["Country", 'ISO 3166-1 alpha-3', 'Year','Continent'], value_vars = ['Coal', 'Oil', 'Gas', 'Cement', 'Flaring'], var_name = "Type", value_name = "Emission")
df_long

,Country,ISO 3166-1 alpha-3,Year,Continent,Type,Emission
0,Afghanistan,AFG,1750,Asia,Coal,NaN
1,Afghanistan,AFG,1751,Asia,Coal,NaN
2,Afghanistan,AFG,1752,Asia,Coal,NaN
3,Afghanistan,AFG,1753,Asia,Coal,NaN
4,Afghanistan,AFG,1754,Asia,Coal,NaN
...,...,...,...,...,...,...
315515,Global,WLD,2017,None,Flaring,391.992176
315516,Global,WLD,2018,None,Flaring,412.115746
315517,Global,WLD,2019,None,Flaring,439.253991
315518,Global,WLD,2020,None,Flaring,407.583673


In [34]:
df_long.to_csv("long_format.csv", float_format = "%.6f")